In [2]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import os
import math

#These are two data sets, one with old cases, one with big 
Legacy = pd.read_csv("./Text/Legacy.csv", encoding= 'unicode_escape')

Modern = pd.read_csv("./Text/Modern.csv", encoding= 'unicode_escape')

Total = Legacy

Total = Total.append(Modern)

In [3]:

#We look at citations, because looking by name of case is unrealiable.
Modern_Selection = Modern["usCite"]
#Drop empty
Modern_Selection = Modern_Selection.dropna()

#Holds case citiations inculding duplicate
Precdent_List_Dups = []

with open('./Text/virgina_v_black.txt') as f:
    for line in f:
        i = 0
        for case_to_check in Modern_Selection:
            #The citiations in cvs needed to be transfromed to be found
            case_to_check_str = str(case_to_check)
            case_to_check_array = case_to_check_str.split("U.S.")

            case_to_check_str = f"{case_to_check_array[0]}U. S.{case_to_check_array[1]}"

            #Add to list
            if case_to_check_str in line:
                #print(f{case_to_check_str})
                Precdent_List_Dups.append(Modern.iloc[i, 14])
                
            i += 1

In [4]:
Precdent_list_caps = []
Precdent_list = []


for i in Precdent_List_Dups:
    if i not in Precdent_list_caps:
        Precdent_list_caps.append(i)

for item in Precdent_list_caps:
    Precdent_list.append(item.title())

#print(Precdent_list)

# Amount of Cases / Time

In [5]:
Decision_Year_dups = Total["term"].values.tolist()

Decision_Year_Count = {i:Decision_Year_dups.count(i) for i in Decision_Year_dups}

Decision_Year_Count_df = pd.DataFrame.from_dict(Decision_Year_Count, orient='index', columns=["Number_Cases"])

#Decision_Year_Count_df

In [6]:
fig = px.line(Decision_Year_Count_df)

fig.update_layout(
    title = "Frequency of Supereme Court Cases by Term (1791-2020)",
    yaxis_title = 'Number of Cases',
    xaxis_title = 'Year',
    showlegend= False,
    xaxis = dict(
        dtick = 10
    )
)

In [7]:

#Name of the coloumn in the data set, your end coloumns, and dictornary
def row_to_df(str_coloum_to_format, columns_for_df, dict_for_df):

    Issue_Term_df = Total[["term", str_coloum_to_format]]

    grouped_df = Issue_Term_df.groupby("term")

    grouped_lists = grouped_df[str_coloum_to_format].apply(list)

    grouped_lists = grouped_lists.reset_index()

    #Format data more
    index =  0

    Final_df = pd.DataFrame(columns = columns_for_df)

    #Some vaules are just not there, we need to count and handle them
    null_values = 0


    #For every year, for every list
    for List in grouped_lists[str_coloum_to_format]:

        list_to_append = []
        str_list_dup = []

        #Determine what issue is at hand, turn to string and add to list
        for float_i in List:
            if math.isnan(float_i):
                null_values += 1
                str_list_dup.append("No Data")
            else:
                int_i = int(float_i)
                str_list_dup.append(dict_for_df[int_i])

        #Count how many issues there are
        count = {i:str_list_dup.count(i) for i in str_list_dup}

        addtive = 0

        #Make an array that has the correct number at the correct position to append.
        for subject in columns_for_df:
            
            int_to_add = 0
            int_to_test = count.get(subject)

            if int_to_test != None:
                int_to_add = int_to_test

            list_to_append.insert(addtive, int_to_add)
            
            addtive += 1

        #Add to df as row
        a_series = pd.Series(list_to_append, index = Final_df.columns)
        Final_df = Final_df.append(a_series, ignore_index=True)

        index += 1


    Final_df = Final_df.set_index(Decision_Year_Count_df.index)

    return Final_df


# Court Cases by Issue

In [8]:

#End Goal is stacked area graph, we have a coloumn issue Area that tells us by number what issue was at hand
#Format the Data to be analyzed by graph

IssueArea_Dict = {
        1:"Criminal Procedure",
        2:"Civil Rights",
        3:"First Amendment",
        4:"Due Process",
        5:"Privacy",
        6:"Attorneys",
        7:"Unions",
        8:"Economic Activity",
        9:"Judicial Power",
        10:"Federalism",
        11:"Interstate Relations",
        12:"Federal Taxation",
        13:"Miscellaneous",
        14:"Private Action"
}
Stacked_Area_df = row_to_df("issueArea", ["No Data", "Criminal Procedure", "Civil Rights", "First Amendment", "Due Process", "Privacy", "Attorneys", "Unions", "Economic Activity", "Judicial Power", "Federalism", "Interstate Relations", "Federal Taxation", "Miscellaneous", "Private Action"], IssueArea_Dict)



#Stacked_Area_df

In [9]:
stacked_area_issue_graph = px.area(Stacked_Area_df)

stacked_area_issue_graph.update_layout(
    title = "Supereme Court Cases by Issue (1791-2020)",
    yaxis_title = 'Number of Cases',
    xaxis_title = 'Term Year',
    legend_title_text='Issue',
    xaxis = dict(
        dtick = 10
    )
)

def stacked_graph_annoations(Year, y_height, text, position):
    stacked_area_issue_graph.add_trace(go.Scatter(
        x=[Year, Year],
        y=[y_height, 0],
        mode="lines+text",
        name=text,
        text=[text],
        textposition=position,
        line=dict(color='LightSeaGreen')
    ))

#Annonations
stacked_graph_annoations(1846, 230, "Start of the <br> Mexican American War", "middle left")
stacked_graph_annoations(1861, 300, "Start of the <br> Civil War", "middle left")
stacked_graph_annoations(1912, 350, "Woodrow Wilson <br> Elected", "middle right")
stacked_graph_annoations(1955, 250, "Vietnam War", "middle right")
stacked_graph_annoations(1941, 275, "US Entered WWII", "middle right")
stacked_graph_annoations(1991, 200, "Soviet Union <br> Dissolution", "top center")
stacked_graph_annoations(1896, 350, "Plessy v. Ferguson", "middle left")
stacked_graph_annoations(1929, 320, "Start of the Great Depression", "middle right")


stacked_area_issue_graph

In [10]:
precdent_dict = {
    0: "False",
    1: "True"
}

precdent_df = row_to_df("precedentAlteration", ["No Data", "True", "False"], precdent_dict)

precdent_df.to_csv("Predent_df.csv")

#px.scatter(precdent_df["True"])

In [11]:
map_dict = {
    1:"AL",
    2:"AK",
    3:"American Samoa",
    4:"AZ",
    5:"AR",
    6:"CA",
    7:"CO",
    8:"CT",
    9:"DE",
    10:"District of Columbia",
    11:"Federated States of Micronesia",
    12:"FL",
    13:"GA",
    14:"Guam",
    15:"HI",
    16:"ID",
    17:"IL",
    18:"IN",
    19:"IA",
    20:"KS",
    21:"KY",
    22:"LA",
    23:"ME",
    24:"Marshall Islands",
    25:"MD",
    26:"MA",
    27:"MI",
    28:"MN",
    29:"MS",
    30:"MO",
    31:"MT",
    32:"NE",
    33:"NV",
    34:"NH",
    35:"NJ",
    36:"NM",
    37:"NY",
    38:"NC",
    39:"ND",
    40:"Northern Mariana Islands",
    41:"OH",
    42:"OK",
    43:"OR",
    44:"Palau",
    45:"PA",
    46:"Puerto Rico",
    47:"RI",
    48:"SC",
    49:"SD",
    50:"TN",
    51:"TX",
    52:"UT",
    53:"VT",
    54:"Virgin Islands",
    55:"VA",
    56:"WA",
    57:"WV",
    58:"WI",
    59:"WY",
    60:"United States",
    61:"Interstate Compact",
    62:"Philippines",
    63:"Indian",
    64:"Dakota"
}

states_not_mappable = ['No Data', 'American Samoa', 'District of Columbia', 'Federated States of Micronesia', 'Guam', 'Marshall Islands', 'Northern Mariana Islands', 'Interstate Compact', 'Philippines', 'Indian', 'Dakota', 'Palau', 'Puerto Rico', 'United States', 'Virgin Islands']

State_Origin_df = row_to_df('caseOriginState', ['No Data', 'AL', 'AK', 'American Samoa', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'District of Columbia', 'Federated States of Micronesia', 'FL', 'GA', 'Guam', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'Marshall Islands', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'Northern Mariana Islands', 'OH', 'OK', 'OR', 'Palau', 'PA', 'Puerto Rico', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'Virgin Islands', 'VA', 'WA', 'WV', 'WI', 'WY', 'United States', 'Interstate Compact', 'Philippines', 'Indian', 'Dakota'], map_dict)

for state in states_not_mappable:
    del State_Origin_df[state]

State_Origin_object_count = State_Origin_df.sum()

State_Origin_df_count = pd.DataFrame(data=State_Origin_object_count, columns=["Number of Cases"])

data_max_min = State_Origin_df_count["Number of Cases"]
max = data_max_min.max(axis=0)
min = data_max_min.min(axis=0)

State_Origin_df_count_log = pd.DataFrame(index = State_Origin_df_count.index)

value_list = []

for value in State_Origin_df_count["Number of Cases"]:
    value = float(value)
    value = math.log(value, 2)
    value_list.append(value)

State_Origin_df_count_log['Value Log'] = 5
State_Origin_df_count_log['Value Log'] = value_list


State_Origin_df_count_log

main_map = px.choropleth(State_Origin_df_count, 
    locations=State_Origin_df_count.index, 
    color = State_Origin_df_count["Number of Cases"].astype(float), 
    locationmode="USA-states", scope="usa", 
    color_continuous_scale = px.colors.sequential.Blues)

#main_map

In [12]:
State_Origin_df_count.sort_values(by=["Number of Cases"],inplace = True, ascending=False)


State_Orgin_Count = px.bar(State_Origin_df_count, x=State_Origin_df_count.index, y = State_Origin_df_count["Number of Cases"])

State_Orgin_Count.update_layout(
    title = "Number of Supereme Court Cases Orginated by State (1790",
    yaxis_title = 'Number of Cases',
    xaxis_title = 'State',
    showlegend= False
)


In [13]:
precdent_df = pd.read_csv("Precdent_alreation_by_issue.csv", encoding= 'unicode_escape')

dict_for_df = IssueArea_Dict

columns_for_df = ["No Data", "Criminal Procedure", "Civil Rights", "First Amendment", "Due Process", "Privacy", "Attorneys", "Unions", "Economic Activity", "Judicial Power", "Federalism", "Interstate Relations", "Federal Taxation", "Miscellaneous", "Private Action"]

Issue_Term_df = precdent_df[["term", "issueArea"]]

grouped_df = Issue_Term_df.groupby("term")

grouped_lists = grouped_df["issueArea"].apply(list)

grouped_lists = grouped_lists.reset_index()

#Format data more
index =  0

Final_df = pd.DataFrame(columns = columns_for_df)

#Some vaules are just not there, we need to count and handle them
null_values = 0


#For every year, for every list
for List in grouped_lists["issueArea"]:

    list_to_append = []
    str_list_dup = []

    #Determine what issue is at hand, turn to string and add to list
    for float_i in List:
        if math.isnan(float_i):
            null_values += 1
            str_list_dup.append("No Data")
        else:
            int_i = int(float_i)
            str_list_dup.append(dict_for_df[int_i])

    #Count how many issues there are
    count = {i:str_list_dup.count(i) for i in str_list_dup}

    addtive = 0

    #Make an array that has the correct number at the correct position to append.
    for subject in columns_for_df:
        
        int_to_add = 0
        int_to_test = count.get(subject)

        if int_to_test != None:
            int_to_add = int_to_test

        list_to_append.insert(addtive, int_to_add)
        
        addtive += 1

    #Add to df as row
    a_series = pd.Series(list_to_append, index = Final_df.columns)
    Final_df = Final_df.append(a_series, ignore_index=True)

    index += 1


Final_df = Final_df.set_index(precdent_df["term"].unique())

Final_df.to_csv("Precdent_Alteration.csv")

In [61]:
precdent_df = pd.read_csv("Precdent_Alteration.csv", index_col=0)

del precdent_df["No Data"]

del precdent_df["Interstate Relations"]

del precdent_df["Miscellaneous"]

precdent_df_sum = precdent_df.sum()

labels = ['Criminal Procedure', 'Civil Rights', 'First Amendment', 'Due Process',
       'Privacy', 'Attorneys', 'Unions', 'Economic Activity', 'Judicial Power',
       'Federalism', 'Federal Taxation', 'Private Action']

vaules = [71, 38, 15, 11, 3, 1, 6, 72, 21, 20, 8, 4]

fig = go.Figure(data=[go.Pie(labels=labels, values=vaules)])
fig.update_traces(textposition='inside')
fig.update_layout(uniformtext_minsize=12, uniformtext_mode='hide')
fig.show()

precdent_df_sum.index


Index(['Criminal Procedure', 'Civil Rights', 'First Amendment', 'Due Process',
       'Privacy', 'Attorneys', 'Unions', 'Economic Activity', 'Judicial Power',
       'Federalism', 'Federal Taxation', 'Private Action'],
      dtype='object')